In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('stocks.csv')

print('Dataset:')
print(data.head())

Dataset:
  Ticker        Date        Open        High         Low       Close  \
0   AAPL  2023-02-07  150.639999  155.229996  150.639999  154.649994   
1   AAPL  2023-02-08  153.880005  154.580002  151.169998  151.919998   
2   AAPL  2023-02-09  153.779999  154.330002  150.419998  150.869995   
3   AAPL  2023-02-10  149.460007  151.339996  149.220001  151.009995   
4   AAPL  2023-02-13  150.949997  154.259995  150.919998  153.850006   

    Adj Close    Volume  
0  154.414230  83322600  
1  151.688400  64120100  
2  150.639999  56007100  
3  151.009995  57450700  
4  153.850006  62199000  


In [ ]:
# Import necessary PyTorch modules 
import torch
import torch.nn as nn
import torch.optim as optim

# Defined the StockNeuralNetwork class with layers and activation functions
class StockNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(StockNeuralNetwork, self).__init__()
        self.first_layer = nn.Linear(input_size, hidden_sizes[0])
        self.relu1 = nn.ReLU()  
        self.second_layer = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)  
        self.output_layer = nn.Linear(hidden_sizes[1], output_size)


# Defined the forward pass logic
    def forward(self, x):
        x = self.first_layer(x)
        x = self.relu1(x)  
        x = self.second_layer(x)
        x = self.relu2(x)  
        x = self.dropout2(x)  
        x = self.output_layer(x)
        return x

In [ ]:
# Set the model parameters(Still testing to see which are the best parameters to use)
"""Model Parameters
input_size =  
hidden_sizes =  
output_size = 
"""
# Instantiate the neural network model
model = StockNeuralNetwork(input_size, hidden_sizes, output_size)

# Defined the loss function and optimizer
criterion = nn.MSELoss()  #
optimizer = optim.Adam(model.parameters(), lr=0.001)